[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?filepath=examples/PyMPDATA_examples/Williamson_and_Rasch_1989_as_in_Jaruga_et_al_2015_Fig_14/demo_over_the_pole.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Williamson_and_Rasch_1989_as_in_Jaruga_et_al_2015_Fig_14/demo_over_the_pole.ipynb)

In [ ]:
# based on MoAC project by Michał Sadowski

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [ ]:
import os
import numpy as np
import numba
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Video
from PyMPDATA import VectorField, ScalarField, Options, Solver, Stepper
from PyMPDATA.boundary_conditions import Periodic, Polar

In [ ]:
options = Options(n_iters=1) #

In [ ]:
nlon = 64 # original: 128
nlat = 32 # original: 64
nt = 5120/3 # original: 5120
dlmb = 2*np.pi / nlon
dphi = np.pi / nlat

r = 5/64*np.pi #  original: 7/64*n.pi
x0 = 3*np.pi / 2
y0 = 0

udt = 2*np.pi / nt
b = -np.pi / 2
h0 = 0

In [ ]:
boundary_conditions = (Periodic(), Polar((nlon, nlat), 0, -1))

In [ ]:
@numba.njit()
def pdf(i, j):
    tmp = 2*(
        (np.cos(dphi * (j + 0.5) - np.pi / 2) * np.sin((dlmb * (i+.5) - x0) / 2))**2 +
        np.sin((dphi * (j + 0.5) - np.pi / 2 - y0) / 2)**2
    )
    return h0 + np.where(
        # if
        tmp - r**2 <= 0,
        # then
        1 - np.sqrt(tmp)/r,
        # else
        0.
    )

In [ ]:
def ad_x(i, j):
    return dlmb * udt * (
        np.cos(b) * np.cos(j * dphi - np.pi / 2) + 
        np.sin(b) * np.sin(j * dphi - np.pi / 2) * np.cos((i+.5) * dlmb)
    )

In [ ]:
def ad_y(i, j):
    return -dlmb * udt * np.sin(b) * np.sin(i * dlmb)* np.cos((j+.5) * dphi - np.pi / 2)

In [ ]:
advector_x = np.array([[
    ad_x(i, j)
    for j in range(nlat)] for i in range(nlon+1)])

advector_y = np.array([[
    ad_y(i, j)
    for j in range(nlat+1)] for i in range(nlon)])

In [ ]:
plt.imshow(advector_x)
plt.colorbar()
print(np.amin(advector_x), np.amax(advector_x))
np.testing.assert_array_almost_equal(advector_x[0,:], advector_x[-1,:])

In [ ]:
plt.imshow(advector_y)
plt.colorbar()
print(np.amin(advector_y), np.amax(advector_y))
np.testing.assert_array_almost_equal(advector_y[:,0], - advector_y[:,-1])

In [ ]:
advector = VectorField(
    data=(advector_x, advector_y),
    halo=options.n_halo,
    boundary_conditions=boundary_conditions    
)

In [ ]:
dx = 1
dy = 1
nx = nlon
ny = nlat

ux, uy = np.mgrid[
    0 : (nx+1)*dx : dx,
    dy/2 : ny*dy : dy
]

vx, vy = np.mgrid[
    dx/2 : nx*dx : dx,
    0: (ny+1)*dy : dy
]

fig = plt.figure(figsize=(15,10))
plt.xticks(ux[:,0])
plt.yticks(vy[0,:])
plt.grid()
plt.quiver(ux, uy, advector.get_component(0), 0, pivot='mid', scale=.025)
plt.quiver(vx, vy, 0, advector.get_component(1), pivot='mid', scale=.025)

In [ ]:
@numba.njit()
def pdf_g_factor(_, y):
    return dlmb * dphi * np.cos(dphi * (y + .5) - np.pi / 2)
g_factor_z = np.array([[
    pdf_g_factor(i, j)
    for j in range(nlat)] for i in range(nlon)])

In [ ]:
plt.imshow(g_factor_z)
plt.colorbar()
print(np.amin(np.abs(g_factor_z)))

In [ ]:
Cx_max = np.amax(np.abs((advector_x[1:,:]+advector_x[:-1,:])/2/g_factor_z))
print(Cx_max)
assert Cx_max < 1

In [ ]:
Cy_max = np.amax(np.abs((advector_y[:,1:]+advector_y[:,:-1])/2/g_factor_z))
print(Cy_max)
assert Cy_max < 1

In [ ]:
g_factor = ScalarField(
    data=g_factor_z, 
    halo=options.n_halo, 
    boundary_conditions=boundary_conditions
)

In [ ]:
z = np.array([[
    pdf(i, j)
    for j in range(nlat)] for i in range(nlon)])

advectee = ScalarField(
    data=z, 
    halo=options.n_halo, 
    boundary_conditions=boundary_conditions
)

In [ ]:
stepper = Stepper(options=options, n_dims=2, non_unit_g_factor=True)

In [ ]:
solver = Solver(stepper=stepper, advectee=advectee, advector=advector, g_factor=g_factor)

In [ ]:
plt.imshow(solver.advectee.get())
plt.colorbar()

In [ ]:
states_history = [z]
for i in range(64):
    solver.advance(n_steps=128)
    states_history.append(solver.advectee.get().copy())    

# Plots 

In [ ]:
from matplotlib import cm
# import matplotlib.animation as animation

theta = np.linspace(0, 1, nlat+1, endpoint=True) * np.pi
phi   = np.linspace(0, 1, nlon+1, endpoint=True) * 2 * np.pi

X = np.outer(np.sin(theta), np.cos(phi))
Y = np.outer(np.sin(theta), np.sin(phi))
Z = np.outer(np.cos(theta), np.ones(nlon+1))

def plot_state(state, save_path=None):
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_axis_off()
    norm = matplotlib.colors.Normalize(vmin=h0, vmax=h0+.05)
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1, 
                    facecolors=cm.copper_r(norm(state.T)), alpha=0.6, linewidth=.75)
    m = cm.ScalarMappable(cmap=cm.copper_r, norm=norm)
    m.set_array([])
    plt.colorbar(m, shrink=.5, ax=ax)
    if save_path:
        plt.savefig(save_path)
    return plt.show()

In [ ]:
!rm -rf animation
!mkdir animation
paths = []
for i, state in enumerate(states_history):
    path = f'animation/{i}.png'
    paths.append(path)
    plot_state(state, path)
    print(np.amin(state), np.amax(state))

In [ ]:
if 'CI' not in os.environ:
    os.system("ffmpeg -f image2 -r 10 -i ./animation/%01d.png -vcodec mpeg4 -y simulation.mp4")

In [ ]:
if 'CI' not in os.environ:
    Video("simulation.mp4")

In [ ]:
# for i in *.png; do convert $i -flatten -trim +repage -bordercolor white -border 5 _$i; done;
# convert _*.png anim.gif